## Libraries


In [ ]:
# !pip install -r ~/code/benitomartin/FoodScore/requirements.txt

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanIoU


import pickle

## Data import

In [2]:
coord = pd.DataFrame()

for i in range(1, 257, 1):
    path = f"../raw_data/UECFOOD256/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [3]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [4]:
coord.shape

(31645, 6)

In [5]:
coord.head()

,img,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


In [6]:
coord = coord.rename(columns={"img": "img_name"})

In [7]:
coord.head()

,img_name,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


### Rescaling and Normalization

In [8]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD256/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [9]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm
0,1,1,0.0000,0.238333,0.46250,0.810000
1,2,1,0.0250,0.346667,0.72750,0.931667
2,3,1,0.0025,0.183333,0.30375,0.683333
3,4,1,0.0000,0.395000,0.35750,0.893333
4,5,1,0.0100,0.046667,0.95125,0.975000


### add image paths

In [10]:
list_paths = [f"../raw_data/UECFOOD256/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [11]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [12]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.46250,0.810000,../raw_data/UECFOOD256/1/1.jpg
1,2,1,0.0250,0.346667,0.72750,0.931667,../raw_data/UECFOOD256/1/2.jpg
2,3,1,0.0025,0.183333,0.30375,0.683333,../raw_data/UECFOOD256/1/3.jpg
3,4,1,0.0000,0.395000,0.35750,0.893333,../raw_data/UECFOOD256/1/4.jpg
4,5,1,0.0100,0.046667,0.95125,0.975000,../raw_data/UECFOOD256/1/5.jpg


In [49]:
rescaled_coord.to_csv('rescaled_coord.csv')

### balancing Dataset

In [13]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [14]:
classes = list(set(rescaled_coord.label))

In [79]:
df = rebalancing(rescaled_coord, classes, av_number= 50, random_state=1)

In [80]:
rescaled_coord[rescaled_coord['label']==100].shape

(104, 7)

In [81]:
df[df['label']==100].shape

(50, 7)

### load downscaled pictures into array

In [82]:
from tqdm.auto import tqdm

In [83]:
df.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
1,2,1,0.025,0.346667,0.727500,0.931667,../raw_data/UECFOOD256/1/2.jpg
18,22,1,0.620,0.533333,1.000000,1.000000,../raw_data/UECFOOD256/1/22.jpg
38,45,1,0.086,0.042667,0.800000,0.941333,../raw_data/UECFOOD256/1/45.jpg
46,61,1,0.000,0.366366,0.456000,0.945946,../raw_data/UECFOOD256/1/61.jpg
47,62,1,0.000,0.256667,0.826667,1.000000,../raw_data/UECFOOD256/1/62.jpg


In [84]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
14969,67342,104,0.168000,0.024000,0.856000,0.744000,../raw_data/UECFOOD256/104/67342.jpg
24586,265664,192,0.020747,0.127778,0.991701,0.769444,../raw_data/UECFOOD256/192/265664.jpg
4674,2708,28,0.059140,0.000000,0.946237,1.000000,../raw_data/UECFOOD256/28/2708.jpg
16947,79246,123,0.058000,0.070000,0.936000,0.936000,../raw_data/UECFOOD256/123/79246.jpg
10652,7585,71,0.000000,0.110000,1.000000,0.790000,../raw_data/UECFOOD256/71/7585.jpg


In [85]:
df_shuffled.to_csv('df_shuffled.csv')

In [86]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

100%|████████████████████████████████████| 12800/12800 [00:45<00:00, 279.06it/s]


In [87]:
type(images)

numpy.ndarray

In [88]:
np.savez_compressed('imgs_2.npz', images)


In [89]:
ls -hal imgs*

-rw-r--r--  1 katharinarolfs  staff    14G Mar 23 09:38 imgs.npz
-rw-r--r--  1 katharinarolfs  staff   2.1G Mar 23 10:47 imgs_2.npz


In [22]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [23]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [24]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [25]:
len(set(df_shuffled.label))

256

In [26]:
tvImages, testImages,tvLabels, testLabels,tvBBoxes, testBBoxes,tvPaths, testPaths=\
train_test_split(images,
                 labels,
                 bboxes,
                 paths,
                 test_size=0.20,
                 random_state=42)

In [27]:
trainImages, valImages,trainLabels, valLabels,trainBBoxes, valBBoxes, trainPaths, valPaths=\
train_test_split(tvImages,
                 tvLabels,
                 tvBBoxes,
                 tvPaths,
                 test_size=0.20,
                 random_state=42)

## Model

In [28]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=layers.Input(shape=(224, 224, 3)))


vgg.trainable = False

flatten = vgg.output
flatten = layers.Flatten()(flatten)

bboxHead = layers.Dense(128, activation="relu")(flatten)
bboxHead = layers.Dense(64, activation="relu")(bboxHead)
bboxHead = layers.Dense(32, activation="relu")(bboxHead)
bboxHead = layers.Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)

softmaxHead = layers.Dense(512, activation="relu")(flatten)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(512, activation="relu")(softmaxHead)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(len(set(df_shuffled.label)), activation="softmax", name="class_label")(softmaxHead)


In [29]:
model = Model(
    inputs=vgg.input,
    outputs=(bboxHead, softmaxHead))

In [30]:
losses = {
    "class_label": 'categorical_crossentropy',
    "bounding_box": "mse"
}

In [31]:
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

In [32]:
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}

In [33]:
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

In [34]:
valTargets = {
    "class_label": valLabels,
    "bounding_box": valBBoxes
}

In [35]:
metrics = {
    "class_label": "categorical_accuracy",
    "bounding_box": MeanIoU(num_classes=len(set(df_shuffled.label)))
}

In [36]:
opt = Adam(0.001)


model.compile(loss=losses, 
              optimizer=opt, 
              metrics=metrics, 
              loss_weights=lossWeights)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [ ]:
es = EarlyStopping(monitor = 'loss',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [ ]:
from datetime import datetime
import keras
logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [47]:
history = model.fit(
    trainImages,
    trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=32,
    epochs=1000,
    verbose=1,
    callbacks = [es, tensorboard_callback],
    )

Epoch 1/1000
256/256 [==============================] - 1177s 5s/step - loss: 5.5587 - bounding_box_loss: 0.0206 - class_label_loss: 5.5381 - bounding_box_mean_io_u: 0.3224 - class_label_categorical_accuracy: 0.0051 - val_loss: 5.5816 - val_bounding_box_loss: 0.0202 - val_class_label_loss: 5.5614 - val_bounding_box_mean_io_u: 0.3212 - val_class_label_categorical_accuracy: 0.0039
Epoch 2/1000
256/256 [==============================] - 1164s 5s/step - loss: 5.5592 - bounding_box_loss: 0.0206 - class_label_loss: 5.5387 - bounding_box_mean_io_u: 0.3224 - class_label_categorical_accuracy: 0.0055 - val_loss: 5.5817 - val_bounding_box_loss: 0.0202 - val_class_label_loss: 5.5615 - val_bounding_box_mean_io_u: 0.3212 - val_class_label_categorical_accuracy: 0.0039
Epoch 3/1000
256/256 [==============================] - 1170s 5s/step - loss: 5.5562 - bounding_box_loss: 0.0206 - class_label_loss: 5.5356 - bounding_box_mean_io_u: 0.3224 - class_label_categorical_accuracy: 0.0055 - val_loss: 5.5819 -

## Save model

In [ ]:
# VGG16

pickle.dump(model, open('vgg16_256classes_50imgs.pkl', 'wb'))

In [ ]:
list(testPaths)

In [48]:
a=3
a

3